In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from sklearn import preprocessing
from sklearn.utils import random as rd

## Cost/activation

In [3]:
def cost_MSE(t,y_hat, derivative=0):
    if derivative:
            return -(t - y_hat)
    return np.mean(1/2*np.sum(np.power(t - y_hat, 2),\
                      axis=0))

def logistic_sigmoid(x, derivative=0):    
    sigm = 1/(1 + np.exp(-x))
    if len(sigm.shape) < 2:
        sigm = sigm.reshape(sigm.shape[0],1)
        
    if derivative:
        return sigm*(1. - sigm)
    return sigm

## NN backend

In [50]:
# >>>>>>>>>>>>>>>>>>> init_weights_biases >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# W, B = init_weights_biases(4, 3, [2,2])
#no_hidden_units: needs a list with at least one element
def init_weights_biases(no_of_features, no_outputs, no_hidden_units, seed=1):
    
    W = []
    B = []
    rows, columns = 0, 0 
    last = len(no_hidden_units)
    np.random.seed(seed)
    
    if no_hidden_units: #list is not empty
        for i in range(last+1):
            if i == 0: #first weight
                rows = no_hidden_units[i]
                columns = no_of_features
            elif i > 0 and i < last:
                rows = no_hidden_units[i]
                columns = no_hidden_units[i-1]
            else: #last
                columns = rows # list ran out of indeces, so use last one
                rows = no_outputs            

            W.insert(i, np.random.randn(rows, columns))
            B.insert(i, np.zeros((rows, 1)))
    else: # no hidden units (perceptron)
        W.insert(0, np.random.randn(no_outputs, no_of_features))
        B.insert(0, np.zeros((no_outputs, 1)))
    
    dummy_param = 0
    param = 0
    for i in range(len(W)):
        dummy_param = W[i].shape[0] * W[i].shape[1]
        param += dummy_param
        
#     W.append(param) #number of learnable weights
    
    return W, B, param

# >>>>>>>>>>>>>>>>>>> forward_prop >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # W1, b1, W2, b2 = init_weights_biases(no_hidden_units=8)
    # Z, A, Y = forward_prop(W, B, X)
    # X has n features x M samples
def forward_prop(W, B, X):
    no_of_samples = X.shape[1]
     #last weight matrix, rows correspond to outputs
    no_of_outputs = W[-2].shape[0] #index -1 is the number of learnable weights
    
    Z = []
    A = []
    A.append(X) #first layer is an activation
    
    for i in range(len(W)): #to avoid the last two indeces
        Z.insert(i, W[i] @ A[i] + B[i])
        A.insert(i+1, logistic_sigmoid(Z[i]))
    
    Y = np.zeros((no_of_samples, no_of_outputs))
    #scaling to making the pair a probability
    Y = np.divide(A[i+1], np.sum(A[i+1], axis=0)) #comuns are the samples now
    return Z, A, Y

# >>>>>>>>>>>>>>>>>>> backprop >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# W1, b1, W2, b2 = init_weights_biases(no_hidden_units=8)
# A1, A2, Y = forward_prop(W1, b1, W2, b2, X)
# grad_mid_layer, grad_output = backprop(W2, A1, A2, X, Y, t)
# backprop(W2, A1, A2, X, Y, t)
def backprop(W, Z, A, Y_hat, T):
    
    output_index = len(W)-1 # if 3, starts at 2
    error = {}
    
    error_output = ( cost_MSE(T,Y_hat, derivative=1) * logistic_sigmoid(Z[-1], derivative=1))
    error[output_index] = error_output
    
    dJ_dW = {}
    for i in range(output_index-1,-1,-1):
         # doesn't get to W[0], so updated after the foor loop again
#         dJ_dW.insert(i+1, error[i+1] @ A[i+1].T)
        dJ_dW[i+1] = error[i+1] @ A[i+1].T
        
        error_dummy = (W[i+1].T @ error[i+1]) * logistic_sigmoid(Z[i], derivative=1)
#         error.insert(i, error_dummy)
        error[i] = error_dummy
    
    dJ_dW[0] = error[0] @ A[0].T
    
    return dJ_dW

## NN frontend 1 

    train, predict functions

In [51]:
# >>>>>>>>>>>>>>>>>>> train >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # all samples (X), 4 x 2, are fed
    # X: dataset, n samples x N features
    # T: binary labels, n labels x L number of ouputs
    # hidden_layers : list with number of neurons for each inner layer.
        # e.g. [3, 4] will yield two layers with 3 and 4 units respectively
    # this function needs n samples > 1 (batch optimization).
def train(X, T, hidden_layers=[2], epochs=500, rho=.1, normalize_data=True, show_cost=0):    
    
    if normalize_data:
        scaler = preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    no_of_features = X.shape[1]
    no_samples = X.shape[0]
    no_outputs = T.shape[1]
    
    W, B, param = init_weights_biases(no_of_features, no_outputs, hidden_layers)
    
    Y_hat = np.zeros((no_outputs, X.shape[0]))
    
    j = 0
    idx_done = 0
    converged = False
    
    ###NESTED function
    def display_NN_info():
        print("* NN ************************************")
        print("   no. inputs (layer 1): " + str(no_of_features) )
        for k in range(len(hidden_layers)):
            print("   layer " + str(k+2) + ": " + str(hidden_layers[k]) + " units")
        print("   output layer ("+ str(k+3) + "): " + str(no_outputs) )
        print("   learnable weights: " + str(param) )
        print("   max epochs: " + "{:,}".format(epochs) )
        print("   learning rate(rho): " + str(rho) )
        
    display_NN_info()
    time.sleep(4)
    
    cost_final = []
    accuracy = []
    
    for i in range(epochs):
        Z, A, Y_hat = forward_prop(W, B, X.T)
        dJ_dW = backprop(W, Z, A, Y_hat, T.T)

        #grad descent
        for j in range(len(W)):
            W[j] = W[j] - rho*dJ_dW[j]

        Y = Y_hat.T

        y_and_T_match = np.allclose(Y, T, rtol=1e-03)        

        if y_and_T_match: #converged
            j += 1 
            if j == 3:
                idx_done = i + 1 # already predicts corretly all the time
            if j > 100: #makes the prediction more robust 
                # ( probability considered 1 == .60 or greater )
                converged = True
                break
    
        cost_final.append(cost_MSE(T, Y_hat.T))
        accuracy.append(calc_accuracy(T, Y_hat.T))
        
        if show_cost:
            print(str(i) + ": accur: "+ str(accuracy[i])+ "%")
            print(" cost: " + str(cost_final[i]))
    
    if show_cost:
        display_NN_info()
        plt.scatter(range(epochs), cost_final, s=1, color="red")
        plt.title("iterations X cost")
        plt.xlabel("iterations")
        plt.ylabel("cost")
    
    
    print("   Start/final Cost: " + "{:.6f}".format(cost_final[0]) \
          + "/" + "{:.6f}".format(cost_final[-1]))
    
    print("   Train start/final accuracy(" + str(no_samples) + " train samples): "\
          + "{:.2f}".format(accuracy[0]) + "/" + "{:.2f}".format(accuracy[-1]) )
    
    return W, B, Y, X, cost_final, epochs, idx_done, converged, rho, normalize_data, accuracy

# >>>>>>>>>>>>>>>>>>> predict >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # X: dataset, n samples x N features
    #  train_pkg: list with [W, B, Y, X, cost_final, epochs, idx_done, converged, rho]
def predict(X, T, train_pkg):
    if len(X.shape) < 2:
        X = X.reshape(1,X.shape[0]) #for one sample
    
    normalized = train_pkg[-1]
    if normalized: #if the data has been normalized
        scaler = preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    Z, A, Y_hat = forward_prop(train_pkg[0], train_pkg[1], X.T)
    
    del Z, A
    return Y_hat.T, calc_accuracy(T, Y_hat.T)

def calc_accuracy(T, Y):
    matches = np.argmax(Y, axis=1) == np.argmax(T, axis=1)
    return len(matches[matches == True])/len(matches)*100

## dataset 2.b and 2.c Iris - includes function to select test and train sets

    T[0:50,0]: setosa
    T[50:100,1]: versicolor
    T[100:150,2]:virginica

In [54]:
df = pd.read_csv('deepanalytics_dataset.csv')
df.head()
y = df['target']
X = df.drop(['data_id','period','target'],axis=1)
print('X', X.head())
print('y', y.head())
X = np.array(X)
y = np.array(y).reshape(len(X),1)
idx_samples = rd.sample_without_replacement(X.shape[0], int(X.shape[0]*.75))

x = X[idx_samples,:]

y_n = np.zeros((y.shape[0],2))
for i in range(y.shape[0]): # creates proabilistic labels
    if y[i] > 0:
        y_n[i,1] = 1
    else:
        y_n[i,0] = 1

idx_tr = rd.sample_without_replacement(X.shape[0], int(X.shape[0]*.50))
x_train = X[idx_samples,:]
y_train = y_n[idx_samples,:]
idx_test = rd.sample_without_replacement(X.shape[0], int(X.shape[0]*.50))
x_test = X[idx_samples,:]
y_test = y_n[idx_samples,:]

# >>>>>>>>>>>>>>>>>>> select_train_test_samples >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def select_train_test_samples(X, T, no_samples_train):
    size = X.shape[0]

    last_index = size-1
    
    all_indeces = np.linspace(0, last_index, size, dtype=np.int16)
    
    samples = []
    labels = []
    idx = []
    random.seed(5)
    idx.append(random.sample(range(size), no_samples_train))
    idx.append(np.delete(all_indeces, idx)) # delete returns a different value every time
    
    for i in range(2):
        samples.insert(i, X[idx[i],:]) # samples[0] is train, [1] are test
        labels.insert(i, T[idx[i],:]) # same for labels
    
    return [samples[0], samples[1], labels[0], labels[1]]

# >>>>>>>>>>>>>>>>>>> train_random_samples >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def train_random_samples(X,T, no_of_train_samples, hidden_layers=[2],\
                         epochs=1000, rho=.1, normalize_data=True, show_cost=0):
    
    no_test_samples = X.shape[0]-no_of_train_samples
    
    print("train/test samples: " + str(no_of_train_samples) + "/" + str(no_test_samples) )
    [X_train, X_test, T_train,T_test] = select_train_test_samples(X, T, no_of_train_samples)
    
    start = time.time()
    train_pkg = train(X_train, T_train, hidden_layers=hidden_layers, epochs=epochs, rho=rho, show_cost=show_cost)
    end = time.time()
    Y_hat, accuracy = predict(X_test, T_test, train_pkg)
    
    print("   Test accuracy(" + str(no_test_samples) +" test samples): " + "{:.2f}".format(accuracy) + "%")
    print('   Time Taken: ', end-start, ' seconds')
    print()
    
    return accuracy, train_pkg

X          c1        c2        c3        c4   c5   c6   c7   c8        c9  c10  \
0 -2.535552  0.000660 -0.000479  0.008932 -0.0  0.0  0.0  0.0 -0.005144 -0.0   
1  0.611008  0.016856 -0.015117 -0.026797 -0.0  0.0  0.0  0.0 -0.065916 -0.0   
2 -0.084158  0.013008 -0.017080 -0.028906 -0.0  0.0  0.0  0.0 -0.035424 -0.0   
3 -1.185109 -0.011085 -0.023120 -0.000092 -0.0  0.0  0.0  0.0 -0.003681 -0.0   
4  0.342375  0.006518  0.003716 -0.010284 -0.0  0.0  0.0  0.0 -0.007461 -0.0   

   ...       c79       c80       c81  c82       c83      c84  c85  c86  \
0  ... -0.121688  0.016361  0.023122  0.0  0.073914  0.00000  0.0  0.0   
1  ... -0.120094  0.057885  0.198569  0.0 -0.084890  0.00000  0.0  0.0   
2  ... -0.009884 -0.025388  0.092343  0.0  0.122077  0.09243  0.0  0.0   
3  ... -0.062372 -0.055605  0.003650  0.0 -0.030415  0.00000  0.0  0.0   
4  ...  0.172567  0.042416 -0.063021  0.0  0.136356  0.00000  0.0  0.0   

        c87       c88  
0 -0.011314  1.055360  
1 -0.021597 -1.882289  


# NN run (ONE HIDDEN LAYER)

The Neural Network can be run in two ways (

    1. using train_random_samples() with parameters will train based on a slice of the data for the train; the other part will be used for test
    2. using train() and then feeding the train package to predict()

In [ ]:
train_pkg = train(x_train, y_train, hidden_layers=[4,8,32,128,32,8,2],
                   epochs=100_000, rho=.01, normalize_data=True, show_cost=1)
predict(x_test, y_test, train_pkg)

* NN ************************************
   no. inputs (layer 1): 88
   layer 2: 4 units
   layer 3: 8 units
   layer 4: 32 units
   layer 5: 128 units
   layer 6: 32 units
   layer 7: 8 units
   layer 8: 2 units
   output layer (9): 2
   learnable weights: 9108
   max epochs: 100,000
   learning rate(rho): 0.01
0: accur: 50.21333333333333%
 cost: 991.9656585534373
1: accur: 49.78666666666667%
 cost: 1250.8258627702667
2: accur: 50.21333333333333%
 cost: 937.8711478557491
3: accur: 50.21333333333333%
 cost: 937.8074476427118
4: accur: 50.21333333333333%
 cost: 937.7609902324732
5: accur: 50.21333333333333%
 cost: 937.7255924934684
6: accur: 50.21333333333333%
 cost: 937.6977234775946
7: accur: 50.21333333333333%
 cost: 937.6752145117757
8: accur: 50.21333333333333%
 cost: 937.6566581224653
9: accur: 50.21333333333333%
 cost: 937.6411003222022
10: accur: 50.21333333333333%
 cost: 937.6278713285836
11: accur: 50.21333333333333%
 cost: 937.6164869671848
12: accur: 50.21333333333333%
 cos

144: accur: 50.21333333333333%
 cost: 937.4774358480981
145: accur: 50.21333333333333%
 cost: 937.4772342536039
146: accur: 50.21333333333333%
 cost: 937.4770319109914
147: accur: 50.21333333333333%
 cost: 937.4768287647521
148: accur: 50.21333333333333%
 cost: 937.476624759518
149: accur: 50.21333333333333%
 cost: 937.4764198400089
150: accur: 50.21333333333333%
 cost: 937.476213950998
151: accur: 50.21333333333333%
 cost: 937.4760070372764
152: accur: 50.21333333333333%
 cost: 937.4757990436265
153: accur: 50.21333333333333%
 cost: 937.4755899147974
154: accur: 50.21333333333333%
 cost: 937.4753795954311
155: accur: 50.21333333333333%
 cost: 937.4751680300853
156: accur: 50.21333333333333%
 cost: 937.4749551631643
157: accur: 50.21333333333333%
 cost: 937.4747409389003
158: accur: 50.21333333333333%
 cost: 937.4745253012989
159: accur: 50.21333333333333%
 cost: 937.4743081941517
160: accur: 50.21333333333333%
 cost: 937.474089560959
161: accur: 50.21333333333333%
 cost: 937.473869344

291: accur: 50.21333333333333%
 cost: 937.3808545479919
292: accur: 50.21333333333333%
 cost: 937.37850363064
293: accur: 50.21333333333333%
 cost: 937.3760780055751
294: accur: 50.21333333333333%
 cost: 937.3735744622784
295: accur: 50.21333333333333%
 cost: 937.3709896000861
296: accur: 50.21333333333333%
 cost: 937.3683198116591
297: accur: 50.21333333333333%
 cost: 937.365561264487
298: accur: 50.21333333333333%
 cost: 937.3627098801132
299: accur: 50.21333333333333%
 cost: 937.3597613107169
300: accur: 50.21333333333333%
 cost: 937.3567109127154
301: accur: 50.21333333333333%
 cost: 937.353553716865
302: accur: 50.21333333333333%
 cost: 937.3502843943721
303: accur: 50.21333333333333%
 cost: 937.3468972181931
304: accur: 50.21333333333333%
 cost: 937.3433860190544
305: accur: 50.21333333333333%
 cost: 937.3397441349343
306: accur: 50.21333333333333%
 cost: 937.3359643532037
307: accur: 50.21333333333333%
 cost: 937.3320388439349
308: accur: 50.21333333333333%
 cost: 937.3279590829

438: accur: 50.21333333333333%
 cost: 937.4754667874885
439: accur: 50.21333333333333%
 cost: 937.475170909846
440: accur: 50.21333333333333%
 cost: 937.4748677845096
441: accur: 50.21333333333333%
 cost: 937.4745570603205
442: accur: 50.21333333333333%
 cost: 937.4742383634245
443: accur: 50.21333333333333%
 cost: 937.4739112953455
444: accur: 50.21333333333333%
 cost: 937.4735754308911
445: accur: 50.21333333333333%
 cost: 937.4732303158712
446: accur: 50.21333333333333%
 cost: 937.4728754644991
447: accur: 50.21333333333333%
 cost: 937.4725103565966
448: accur: 50.21333333333333%
 cost: 937.472134434443
449: accur: 50.21333333333333%
 cost: 937.4717470992889
450: accur: 50.21333333333333%
 cost: 937.4713477074561
451: accur: 50.21333333333333%
 cost: 937.4709355660087
452: accur: 50.21333333333333%
 cost: 937.4705099278902
453: accur: 50.21333333333333%
 cost: 937.4700699864757
454: accur: 50.21333333333333%
 cost: 937.4696148694857
455: accur: 50.21333333333333%
 cost: 937.46914363

585: accur: 50.21333333333333%
 cost: 906.675390960819
586: accur: 50.21333333333333%
 cost: 919.9956300523534
587: accur: 52.04%
 cost: 937.1234441606377
588: accur: 50.973333333333336%
 cost: 935.8867612375262
589: accur: 49.06666666666666%
 cost: 930.3155797618581
590: accur: 50.21333333333333%
 cost: 916.290772004777
591: accur: 50.21333333333333%
 cost: 904.2635361068146
592: accur: 50.21333333333333%
 cost: 902.2618797315263
593: accur: 50.21333333333333%
 cost: 901.3350032866949
594: accur: 50.21333333333333%
 cost: 900.8555855486011
595: accur: 50.21333333333333%
 cost: 906.0196852749604
596: accur: 50.21333333333333%
 cost: 915.6635393124575
597: accur: 53.54666666666667%
 cost: 933.9168610750172
598: accur: 55.333333333333336%
 cost: 924.2368539680512
599: accur: 50.21333333333333%
 cost: 906.8355156564303
600: accur: 50.21333333333333%
 cost: 902.7562579341253
601: accur: 56.18666666666666%
 cost: 907.1371266085696
602: accur: 50.21333333333333%
 cost: 905.0880336147676
603:

...

1200: accur: 65.32%
 cost: 828.9243427938795
1201: accur: 64.8%
 cost: 834.5334963590551
1202: accur: 64.2%
 cost: 844.1949640541803
1203: accur: 65.10666666666667%
 cost: 832.9593136043266
1204: accur: 64.89333333333333%
 cost: 834.1177859361965
1205: accur: 64.93333333333334%
 cost: 832.0971765443684
1206: accur: 64.85333333333332%
 cost: 832.4830123019315
1207: accur: 64.56%
 cost: 839.6647146848982
1208: accur: 64.86666666666666%
 cost: 835.8450617600768
1209: accur: 64.4%
 cost: 843.0372710014891
1210: accur: 64.85333333333332%
 cost: 833.2916079684079
1211: accur: 65.09333333333333%
 cost: 831.4535935807633
1212: accur: 65.0%
 cost: 830.329203955558
1213: accur: 65.33333333333333%
 cost: 828.1032347333091
1214: accur: 64.48%
 cost: 837.5078369468798
1215: accur: 63.56%
 cost: 860.3556244687231
1216: accur: 64.26666666666667%
 cost: 843.9105794976008
1217: accur: 64.72%
 cost: 835.2592985616553
1218: accur: 65.13333333333333%
 cost: 828.6809199527395
1219: accur: 65.28%
 cost: 830

1353: accur: 61.86666666666667%
 cost: 877.7484956944924
1354: accur: 62.653333333333336%
 cost: 861.409456970693
1355: accur: 62.61333333333333%
 cost: 861.0622157656915
1356: accur: 63.10666666666667%
 cost: 856.525649379011
1357: accur: 63.26666666666667%
 cost: 854.0267687887815
1358: accur: 63.519999999999996%
 cost: 851.2790020362592
1359: accur: 63.480000000000004%
 cost: 851.7107743505144
1360: accur: 63.906666666666666%
 cost: 848.1207966427039
1361: accur: 63.693333333333335%
 cost: 850.6435768595709
1362: accur: 63.33333333333333%
 cost: 852.5702753100861
1363: accur: 63.959999999999994%
 cost: 847.9054395615572
1364: accur: 64.08%
 cost: 846.1952173660922
1365: accur: 63.92%
 cost: 847.3600970393461
1366: accur: 63.866666666666674%
 cost: 849.017516198352
1367: accur: 63.906666666666666%
 cost: 849.26343534737
1368: accur: 63.54666666666666%
 cost: 854.748068216439
1369: accur: 63.480000000000004%
 cost: 855.3742472377571
1370: accur: 64.0%
 cost: 849.8791772268979
1371: ac

1503: accur: 63.05333333333333%
 cost: 857.0005704267278
1504: accur: 62.50666666666667%
 cost: 863.6216624001836
1505: accur: 63.013333333333335%
 cost: 858.5076497499474
1506: accur: 63.26666666666667%
 cost: 856.5562512314546
1507: accur: 63.2%
 cost: 857.2056215692544
1508: accur: 62.906666666666666%
 cost: 860.9953074994783
1509: accur: 63.53333333333333%
 cost: 855.3010586517775
1510: accur: 63.32%
 cost: 856.299483684659
1511: accur: 63.63999999999999%
 cost: 855.8824734253012
1512: accur: 63.22666666666667%
 cost: 859.4371562237666
1513: accur: 61.68%
 cost: 881.7948557329241
1514: accur: 62.72%
 cost: 868.5123702074666
1515: accur: 63.22666666666667%
 cost: 860.4976297518931
1516: accur: 63.346666666666664%
 cost: 860.0472083387666
1517: accur: 62.78666666666667%
 cost: 869.2096089688081
1518: accur: 62.760000000000005%
 cost: 867.4636139840158
1519: accur: 63.4%
 cost: 859.6188683364544
1520: accur: 63.46666666666667%
 cost: 858.5352464998292
1521: accur: 63.21333333333333%
 

1656: accur: 56.99999999999999%
 cost: 909.3153280789129
1657: accur: 57.013333333333335%
 cost: 909.2537470937452
1658: accur: 57.013333333333335%
 cost: 909.1935894539291
1659: accur: 57.02666666666667%
 cost: 909.1347341069215
1660: accur: 57.04%
 cost: 909.0770960391974
1661: accur: 57.053333333333335%
 cost: 909.0206166059388
1662: accur: 57.09333333333333%
 cost: 908.9652543797683
1663: accur: 57.09333333333333%
 cost: 908.910977569565
1664: accur: 57.09333333333333%
 cost: 908.857758490237
1665: accur: 57.08%
 cost: 908.8055700345391
1666: accur: 57.08%
 cost: 908.7543836556549
1667: accur: 57.09333333333333%
 cost: 908.704168055091
1668: accur: 57.120000000000005%
 cost: 908.6548876428385
1669: accur: 57.120000000000005%
 cost: 908.6064999563062
1670: accur: 57.120000000000005%
 cost: 908.5589516192803
1671: accur: 57.120000000000005%
 cost: 908.5121730372108
1672: accur: 57.120000000000005%
 cost: 908.4660727154848
1673: accur: 57.120000000000005%
 cost: 908.4205326912038
1674

1806: accur: 57.41333333333334%
 cost: 904.6645450200615
1807: accur: 57.41333333333334%
 cost: 904.6469587013421
1808: accur: 57.41333333333334%
 cost: 904.629406080026
1809: accur: 57.41333333333334%
 cost: 904.6118849483289
1810: accur: 57.41333333333334%
 cost: 904.5943932952349
1811: accur: 57.41333333333334%
 cost: 904.576929298129
1812: accur: 57.41333333333334%
 cost: 904.5594913123879
1813: accur: 57.42666666666667%
 cost: 904.542077858552
1814: accur: 57.42666666666667%
 cost: 904.5246876069443
1815: accur: 57.42666666666667%
 cost: 904.5073193595213
1816: accur: 57.41333333333334%
 cost: 904.4899720290991
1817: accur: 57.41333333333334%
 cost: 904.4726446159171
1818: accur: 57.4%
 cost: 904.4553361819944
1819: accur: 57.4%
 cost: 904.4380458233301
1820: accur: 57.4%
 cost: 904.4207726406463
1821: accur: 57.4%
 cost: 904.4035157090061
1822: accur: 57.4%
 cost: 904.3862740468447
1823: accur: 57.4%
 cost: 904.3690465850143
1824: accur: 57.4%
 cost: 904.3518321362849
1825: accur

# 2.c. NN run (TWO LAYERS-DEEP)

The Neural Network can be run in two ways (

    1. using train_random_samples() with parameters will train based on a slice of the data for the train; the other part will be used for test
    2. using train() and then feeding the train package to predict()

In [8]:
    
# length_smallest
# [50, 75, 100, 125]
accuracys = {}  #dictionary indexed with number of train samples used
train_pkgs = {} #same as previous

for i in range(len(no_train_samples)):
    accuracys[no_train_samples[i]], train_pkgs[no_train_samples[i]] = \
                            train_random_samples(X,T, no_train_samples[i], hidden_layers=[1,3],\
                                     epochs=5_000, rho=.1, normalize_data=True)

TypeError: object of type 'int' has no len()

In [9]:


# length = X.shape[0]


In [22]:
x.shape

(7500, 88)

In [28]:
y_train

array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [61]:
y_train

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])